# Pre-trained ResNet을 Fine tuning한 버섯 이미지 classification

##1. Setup

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.datasets import ImageFolder
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import pandas as pd


# Set the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


##2. Data Load & Preprocess

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Pre-process the data
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [6]:
# Load all the data
data_root = '/content/drive/MyDrive/Study/ML_TeamProject/mushroom_data_new'
batch_size = 32


# 1. Load all data using ImageFolder
full_dataset = ImageFolder(data_root, transform=transform)

# 2. Calculate lengths for train, validation, and test splits
total_size = len(full_dataset)
train_size = int(0.7 * total_size)
val_size = int(0.2 * total_size)
test_size = total_size - train_size - val_size

# 3. Use random_split to split the datasets
train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])

def set_transform(dataset_subset, transform):
    dataset_subset.dataset.transform = transform
    return dataset_subset

val_dataset = set_transform(val_dataset, transform)
test_dataset = set_transform(test_dataset, transform)

# 4. Create DataLoaders for each set
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
testloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

len(trainloader)


279


##3. Pre-Trained ResNet Load

In [7]:
resnet = models.resnet50(weights='IMAGENET1K_V1')

# Replace the final classification layer
num_features = resnet.fc.in_features
num_classes = len(full_dataset.classes)
resnet.fc = nn.Linear(num_features, num_classes)

resnet = resnet.to(device)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 118MB/s]


##4. Define the Loss function and Optimizer

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet.parameters(), lr=0.005, momentum=0.9)
scheduler = StepLR(optimizer, step_size=10, gamma=0.1)


4. Train the network

In [10]:
from tqdm.notebook import tqdm

num_epochs = 50
print_term = 10


for epoch in range(num_epochs):
    resnet.train()

    running_loss = 0.0
    correct_train = 0
    total_train = 0

    pbar = tqdm(trainloader, total=len(trainloader))
    for batch in pbar:

        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        # 순전파
        outputs = resnet(inputs)
        _, predicted = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels)

        # 역전파 + 최적화
        loss.backward()
        optimizer.step()

        # 통계
        running_loss += loss.item()
        correct_train += (predicted == labels).sum().item()
        total_train += labels.size(0)

    train_loss = running_loss / len(trainloader.dataset)
    train_accuracy = 100 * correct_train / total_train
    pbar.set_description(f"Epoch {epoch+1}/{num_epochs}, Loss: {train_loss.item():.4f}, Train Acc: {train_accuracy:.2f}%")

    scheduler.step()

   # Validation loop
    resnet.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for data in valloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = resnet(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_accuracy = 100 * correct_val / total_val
    average_val_loss = val_loss / len(valloader.dataset)

    print(f"Epoch {epoch+1}/{num_epochs}, Validation loss: {average_val_loss:.3f}, Validation accuracy: {val_accuracy:.2f}%")

print('Finished Training')


  0%|          | 0/279 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
# Save the trained model
torch.save(resnet.state_dict(), 'resnet_model.pth')

In [ ]:
# Assuming the model's class is `Net` as in previous examples
model = torch.load('resnet_model.pth')
model.eval()  # Set the model to evaluation mode


Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

6. Test the network on test data

In [ ]:
# correct = 0
# total = 0
# with torch.no_grad():
#     for data in testloader:
#         images, labels = data[0].to(device), data[1].to(device)
#         outputs = model(images)
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

# print('Accuracy of the network on the 10000 test images: %d %%' % (
#     100 * correct / total))


Accuracy of the network on the 10000 test images: 58 %
